In [1]:
import os
import numpy as np
import torch


from mm_interface import master
from mm_interface.train_test import test_models
from core.utils.randomseed_config import randomseed_config
from core.utils.master import create_output_dirs
from MODELS.loss_functions.get_loss_function import get_lossFun
# from mm_interface.train_test import test_differentiable_model, test_dp_hbv

from hydroDL.master.master import calFDC
from hydroDL.post.stat import statError

import warnings
warnings.filterwarnings("ignore")



# Settings for personal run.
MODELS = ('dPLHBV_dyn', 'SACSMA_snow', 'marrmot_PRMS')  # HBV, dPLHBV_stat, dPLHBV_dyn, SACSMA, SACSMA_snow, PRMS 
ENSEMBLE_TYPE = 'max'  # max, median, avg, ensemble_softmax, ensemble_sigmoid, mosaid


# Setting globals, dirs, randomseed
device, dtype = master.set_globals()
randomseed_config(0)
out_dir = master.set_platform_dir()  # or set your alt path to model results here.

# Setting dictionaries to separately manage each model's attributes.
models, args_list = master.get_model_dict(MODELS)


loading package hydroDL
Setting seed 0


In [2]:
# Forward individual hydro models and get their predictions.
preds, y_obs, models = test_models(models, args_list, out_dir)

daymet tmean was used!
read usgs streamflow 20.615614414215088
read usgs streamflow 20.214037656784058
daymet tmean was used!
read usgs streamflow 19.95155096054077
read usgs streamflow 19.89000153541565
daymet tmean was used!
read usgs streamflow 19.91431212425232
read usgs streamflow 19.820414066314697
read usgs streamflow 24.167844533920288
read master file D:\code_repos\water\data\model_runs\rnnStreamflow\CAMELSDemo/dPLHBV/ALL/TDTestforc/TD1_13/daymet/BuffOpt0/RMSE_para0.25/111111\Fold1\T_19801001_19951001_BS_100_HS_256_RHO_365_NF_13_Buff_365_Mul_16\master.json
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
Setting seed 0


100%|██████████| 27/27 [06:45<00:00, 15.01s/Batch]


Setting seed 0


100%|██████████| 27/27 [04:04<00:00,  9.05s/Batch]


In [31]:
# path = os.path.join(out_dir, 'hydro_models', '671_sites_dp', 'merged_' + 'test2' + '_preds_obs')
# master.save_output(args_list, preds, y_obs, out_dir=path)


# If you've already run the trained models on train/test data, you can import
# the set of model predictions and observed streamflows to save some time.
pred_path = os.path.join(out_dir, 'hydro_models', '671_sites_dp', 'merged_train_preds_obs', 'HBV_dynamic_SACSMA_snow_marrmot_PRMS', 'preds_multim_E50_B25_R365_BT365_H256_tr1980_1995_n16.npy')
obs_path = os.path.join(out_dir, 'hydro_models', '671_sites_dp', 'merged_train_preds_obs', 'HBV_dynamic_SACSMA_snow_marrmot_PRMS', 'obs_multim_E50_B25_R365_BT365_H256_tr1980_1995_n16.npy')
preds = np.load(pred_path, allow_pickle=True).item()
y_obs = np.load(obs_path, allow_pickle=True).item()

preds['dPLHBV_dyn'] = preds.pop('HBV_dynamic')
y_obs['dPLHBV_dyn'] = y_obs.pop('HBV_dynamic')

In [34]:
# Initializing
flow_preds = []
flow_obs = None
obs_trig = False

# Concatenate individual model predictions, and observation data.
for i, mod in enumerate(args_list):
    args = args_list[mod]
    mod_out = preds[mod]
    y_ob = y_obs[mod]

    print(mod)

    if mod in ['HBV', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']:
        # Hydro models are tested in batches, so we concatenate them and select
        # the desired flow.
        # Note: modified HBV already has this preparation done during testing.

        # Get flow predictions and swap axes to get shape [basins, days]
        pred = np.swapaxes(torch.cat([d["flow_sim"] for d in mod_out], dim=1).squeeze().numpy(), 0, 1)

        if obs_trig == False:
            # dPLHBV uses GAGES while the other hydro models use CAMELS data. This means small
            # e-5 variation in observation data between the two. This is averaged if both models
            # are used, but to avoid double-counting data from multiply hydro models, use a trigger.
            obs = np.swapaxes(y_ob[:, :, args["target"].index("00060_Mean")].numpy(), 0, 1)
            obs_trig = True
            dup = False
        else:
            dup = True

    elif mod in ['dPLHBV_stat', 'dPLHBV_dyn']:
        pred = mod_out[:,:,0][:,365:] # Set dim2 = 0 to get streamflow Qr
        obs = y_ob.squeeze()[:,365:]
        dup = False

    else:
        raise ValueError(f"Unsupported model type in `models`.")

    if i == 0:
        tmp_pred = pred
        tmp_obs = obs
    elif i == 1:
        tmp_pred = np.stack((tmp_pred, pred), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.stack((tmp_obs, obs), axis=2)
    else:
        # Combine outputs of >3 models.
        tmp_pred = np.concatenate((tmp_pred,np.expand_dims(pred, 2)), axis=2)
        if not dup:
            # Avoid double-counting GAGES obs.
            tmp_obs = np.concatenate((tmp_obs,np.expand_dims(obs, 2)), axis=2)

preds = tmp_pred
obs = tmp_obs

# Merge observation data.
if len(obs.shape) == 3:
    comp_obs = np.mean(obs, axis = 2)
elif len(obs.shape) == 2:
    comp_obs = obs
else:
    raise ValueError("Error reading prediction data: incorrect formatting.")


dPLHBV_dyn
SACSMA_snow
marrmot_PRMS


## Run Weighting LSTM

In [35]:
import sys
sys.path.append('../../')
from hydroDL import master, utils
from hydroDL.data import camels
from hydroDL.master import default
from hydroDL.master.master import loadModel
from hydroDL.model import rnn, crit, train


import os
import numpy as np
import torch
from collections import OrderedDict
import random
import json
import datetime as dt

## fix the random seeds for reproducibility
randomseed = 111111
random.seed(randomseed)
torch.manual_seed(randomseed)
np.random.seed(randomseed)
torch.cuda.manual_seed(randomseed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## GPU setting
# which GPU to use when having multiple
traingpuid = 0
# torch.cuda.set_device(traingpuid)

## Setting training options here
PUOpt = 0
# PUOpt values and explanations:
# 0: train and test on ALL basins;
# 1 for PUB spatial test, randomly hold out basins;
# 2 for PUR spatial test, hold out a continuous region;
buffOpt = 1
# buffOpt defines the warm-up option for the first year of training forcing data
# 0: do nothing, the first year forcing would only be used to warm up the next year;
# 1: repeat first year forcing to warm up the first year;
# 2: load one more year forcing to warm up the first year
TDOpt = True
# TDOpt, True as using dynamic parameters and False as using static parameters
forType = 'daymet'
# for Type defines which forcing in CAMELS to use: 'daymet', 'nldas', 'maurer'


# used for multimodel ensembling.
prcp_loss_factor = 23
smooth_loss_factor = 0

## Set hyperparameters
EPOCH = 50 # total epoches to train the mode
BATCH_SIZE = 100
RHO = 365
HIDDENSIZE = 256
saveEPOCH = 10
Ttrain = [19801001, 19951001] # Training period
# Ttrain = [19891001, 19991001] # PUB/PUR period
Tinv = [19801001, 19951001] # Inversion period for historical forcings
# Tinv = [19891001, 19991001] # PUB/PUR period
Nfea = 12 # number of HBV parameters. 12:original HBV; 13:includes the added dynamic ET para when setting ETMod=True
BUFFTIME = 365 # for each training sample, to use BUFFTIME days to warm up the states.
routing = True # Whether to use the routing module for simulated runoff
Nmul = 16 # Multi-component model. How many parallel HBV components to use. 1 means the original HBV.
comprout = False # True is doing routing for each component
compwts = False # True is using weighted average for components; False is the simple mean
pcorr = None # or a list to give the range of precip correction

flow_regime = 0  # 1 is high flow expert.

if TDOpt is True:
    # Below options are only for running models with dynamic parameters
    tdRep = [1, 13] # When using dynamic parameters, this list defines which parameters to set as dynamic
    tdRepS = [str(ix) for ix in tdRep]
    # ETMod: if True, use the added shape parameter (index 13) for ET. Default as False.
    # Must set below ETMod as True and Nfea=13 when including 13 index in above tdRep list for dynamic parameters
    # If 13 not in tdRep list, set below ETMod=False and Nfea=12 to use the original HBV without ET shape para
    ETMod = True
    Nfea = 13 # should be 13 when setting ETMod=True. 12 when ETMod=False
    dydrop = 0.0 # dropout possibility for those dynamic parameters: 0.0 always dynamic; 1.0 always static
    staind = -1 # which time step to use from the learned para time series for those static parameters
    TDN = '/TDTestforc/'+'TD'+"_".join(tdRepS) +'/'
else:
    TDN = '/Testforc/'

# Define root directory of database and output
# Modify these based on your own location of CAMELS dataset
# Following the data download instruction in README file, you should organize the folders like
# 'your/path/to/Camels/basin_timeseries_v1p2_metForcing_obsFlow' and 'your/path/to/Camels/camels_attributes_v2.0'
# Then 'rootDatabase' here should be 'your/path/to/Camels';
# 'rootOut' is the root dir where you save the trained model



# CAMELS dataset root directory
# sysroot = '/Users/leoglonz/Desktop/water/data'
# sysroot = '/content/'
sysroot = 'D:\code_repos\water\data'
rootDatabase = os.path.join(os.path.sep, sysroot, 'Camels')    # CAMELS dataset root directory
# rootDatabase = os.path.join(os.path.sep, 'data', 'kas7897', 'dPLHBVrelease')  # CAMELS dataset root directory
camels.initcamels(flow_regime, forType=forType, rootDB=rootDatabase)  # initialize camels module-scope variables in camels.py (dirDB, gageDict) to read basin info

# rootOut = os.path.join(os.path.sep, 'data', 'rnnStreamflow')  # Model output root directory
rootOut = os.path.join(os.path.sep, sysroot, 'data', 'model_runs', 'hydro_multimodel_results', 'multimodels', '671_sites_dp', 'mm_ensemble_sigmoid')  # Model output root directory



## If you have a checkpoint file for a previous run, set the path to directory here.
# saved epoch is the epoch corresponding to the checkpoint.
saved_epoch = 6
checkpoint_file = None #'/Users/leoglonz/Desktop/water/data/model_runs/rnnStreamflow/CAMELSDemo/dPLHBV/ALL/TDTestforc/TD1_13/daymet/BuffOpt0/RMSE_para0.25/111111/Fold1/T_19801001_19951001_BS_100_HS_256_RHO_365_NF_13_Buff_365_Mul_16/'

## set up different data loadings for ALL, PUB, PUR
testfoldInd = 1
# Which fold to hold out for PUB (10 folds, from 1 to 10) and PUR (7 folds, from 1 to 7).
# It doesn't matter when training on ALL basins (setting PUOpt=0), could always set testfoldInd=1 for this case.

# load CAMELS basin information
gageinfo = camels.gageDict
hucinfo = gageinfo['huc']
gageid = gageinfo['id']
gageidLst = gageid.tolist()

if PUOpt == 0: # training on all basins without spatial hold-out
    puN = 'ALL'
    TrainLS = gageidLst # all basins
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    TestLS = gageidLst
    TestInd = [gageidLst.index(j) for j in TestLS]
    gageDic = {'TrainID':TrainLS, 'TestID':TestLS}

elif PUOpt == 1: # random hold out basins. hold out the fold set by testfoldInd
    puN = 'PUB'
    # load the PUB basin groups
    # randomly divide CAMELS basins into 10 groups and this file contains the basin ID for each group
    # located in splitPath
    splitPath = 'PUBsplitLst.txt'
    with open(splitPath, 'r') as fp:
        testIDLst=json.load(fp)
    # Generate training ID lists excluding the hold out fold
    TestLS = testIDLst[testfoldInd - 1]
    TestInd = [gageidLst.index(j) for j in TestLS]
    TrainLS = list(set(gageid.tolist()) - set(TestLS))
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    gageDic = {'TrainID':TrainLS, 'TestID':TestLS}

elif PUOpt == 2:
    puN = 'PUR'
    # Divide CAMELS dataset into 7 continous PUR regions, as shown in Feng et al, 2021 GRL; 2022 HESSD
    # get the id list of each PUR region, save to list
    regionID = list()
    regionNum = list()
    # seven regions including different HUCs
    regionDivide = [ [1,2], [3,6], [4,5,7], [9,10], [8,11,12,13], [14,15,16,18], [17] ]
    for ii in range(len(regionDivide)):
        tempcomb = regionDivide[ii]
        tempregid = list()
        for ih in tempcomb:
            tempid = gageid[hucinfo==ih].tolist()
            tempregid = tempregid + tempid
        regionID.append(tempregid)
        regionNum.append(len(tempregid))

    iexp = testfoldInd - 1  #index
    TestLS = regionID[iexp] # basin ID list for testing, hold out for training
    TestInd = [gageidLst.index(j) for j in TestLS]
    TrainLS = list(set(gageid.tolist()) - set(TestLS)) # basin ID for training
    TrainInd = [gageidLst.index(j) for j in TrainLS]
    gageDic = {'TrainID': TrainLS, 'TestID': TestLS}


# apply buffOPt to solve the warm-up for the first year
if buffOpt ==2: # load more BUFFTIME data for the first year
    sd = utils.time.t2dt(Ttrain[0]) - dt.timedelta(days=BUFFTIME)
    sdint = int(sd.strftime("%Y%m%d"))
    TtrainLoad = [sdint, Ttrain[1]]
    TinvLoad = [sdint, Ttrain[1]]
else:
    TtrainLoad = Ttrain
    TinvLoad = Tinv

## prepare input data
## load camels dataset
if forType == 'daymet':
    varF = ['prcp', 'tmean']
    varFInv = ['prcp', 'tmean']
else:
    varF = ['prcp', 'tmax'] # For CAMELS maurer and nldas forcings, tmax is actually tmean
    varFInv = ['prcp', 'tmax']

# the attributes used to learn parameters
attrnewLst = [ 'p_mean','pet_mean','p_seasonality','frac_snow','aridity','high_prec_freq','high_prec_dur',
               'low_prec_freq','low_prec_dur', 'elev_mean', 'slope_mean', 'area_gages2', 'frac_forest', 'lai_max',
               'lai_diff', 'gvf_max', 'gvf_diff', 'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50',
               'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity',
               'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'geol_1st_class', 'glim_1st_class_frac',
               'geol_2nd_class', 'glim_2nd_class_frac', 'carbonate_rocks_frac', 'geol_porostiy', 'geol_permeability']

optData = default.optDataCamels # a default dictionary for logging, updated below
# Update the training period and variables
optData = default.update(optData, tRange=TtrainLoad, varT=varFInv, varC=attrnewLst, subset=TrainLS, forType=forType)

# for HBV model training inputs
dfTrain = camels.DataframeCamels(tRange=TtrainLoad, subset=TrainLS, forType=forType)
forcUN = dfTrain.getDataTs(varLst=varF, doNorm=False, rmNan=False, flow_regime=flow_regime)
obsUN = dfTrain.getDataObs(doNorm=False, rmNan=False, basinnorm=False, flow_regime=flow_regime)

# for dPL inversion data, inputs of gA
dfInv = camels.DataframeCamels(tRange=TinvLoad, subset=TrainLS, forType=forType)
forcInvUN = dfInv.getDataTs(varLst=varFInv, doNorm=False, rmNan=False, flow_regime=flow_regime)
attrsUN = dfInv.getDataConst(varLst=attrnewLst, doNorm=False, rmNan=False, flow_regime=flow_regime)

# Unit transformation, discharge obs from ft3/s to mm/day
areas = gageinfo['area'][TrainInd] # unit km2
temparea = np.tile(areas[:, None, None], (1, obsUN.shape[1],1))
obsUN = (obsUN * 0.0283168 * 3600 * 24) / (temparea * (10 ** 6)) * 10**3 # transform to mm/day

# load potential ET calculated by hargreaves method
varLstNL = ['PEVAP']
usgsIdLst = gageid
if forType == 'maurer':
    tPETRange = [19800101, 20090101]
else:
    tPETRange = [19800101, 20150101]
tPETLst = utils.time.tRange2Array(tPETRange)

# Modify this as the directory where you put PET
PETDir = rootDatabase + '/pet_harg/' + forType + '/'
ntime = len(tPETLst)
PETfull = np.empty([len(usgsIdLst), ntime, len(varLstNL)])
for k in range(len(usgsIdLst)):
    dataTemp = camels.readcsvGage(PETDir, usgsIdLst[k], varLstNL, ntime)
    PETfull[k, :, :] = dataTemp

TtrainLst = utils.time.tRange2Array(TtrainLoad)
TinvLst = utils.time.tRange2Array(TinvLoad)
C, ind1, ind2 = np.intersect1d(TtrainLst, tPETLst, return_indices=True)
PETUN = PETfull[:, ind2, :]
PETUN = PETUN[TrainInd, :, :] # select basins
C, ind1, ind2inv = np.intersect1d(TinvLst, tPETLst, return_indices=True)
PETInvUN = PETfull[:, ind2inv, :]
PETInvUN = PETInvUN[TrainInd, :, :]

# process data, do normalization and remove nan
series_inv = np.concatenate([forcInvUN, PETInvUN], axis=2)
seriesvarLst = varFInv + ['pet']
# calculate statistics for normalization and saved to a dictionary
statDict = camels.getStatDic(attrLst=attrnewLst, attrdata=attrsUN, seriesLst=seriesvarLst, seriesdata=series_inv, flow_regime=flow_regime)
# normalize data
attr_norm = camels.transNormbyDic(attrsUN, attrnewLst, statDict, toNorm=True, flow_regime=flow_regime)
attr_norm[np.isnan(attr_norm)] = 0.0
series_norm = camels.transNormbyDic(series_inv, seriesvarLst, statDict, toNorm=True, flow_regime=flow_regime)
series_norm[np.isnan(series_norm)] = 0.0

# prepare the inputs
zTrain = series_norm # used as the inputs for dPL inversion gA along with attributes
xTrain = np.concatenate([forcUN, PETUN], axis=2) # used as HBV forcing
xTrain[np.isnan(xTrain)] = 0.0

if buffOpt == 1: # repeat the first year warm up the first year itself
    zTrainIn = np.concatenate([zTrain[:,0:BUFFTIME,:], zTrain], axis=1)
    xTrainIn = np.concatenate([xTrain[:,0:BUFFTIME,:], xTrain], axis=1) # repeat forcing to warm up the first year
    yTrainIn = np.concatenate([obsUN[:,0:BUFFTIME,:], obsUN], axis=1)
else: # no repeat, original data, the first year data would only be used as warmup for the next following year
    zTrainIn = zTrain
    xTrainIn = xTrain
    yTrainIn = obsUN

forcTuple = (xTrainIn, zTrainIn)
attrs = attr_norm

## Train the model
# define loss function
alpha = 0.25 # a weight for RMSE loss to balance low and peak flow
optLoss = default.update(default.optLossComb, name='hydroDL.model.crit.RmseLossComb', weight=alpha)
lossFun = crit.RmseLossComb(alpha=alpha)

# define training options
optTrain = default.update(default.optTrainCamels, miniBatch=[BATCH_SIZE, RHO], nEpoch=EPOCH, saveEpoch=saveEPOCH)
# define output folder to save model results
exp_name = 'CAMELSDemo'
exp_disp = 'dPLHBV/' + puN + TDN + forType + '/BuffOpt'+str(buffOpt)+'/RMSE_para'+str(alpha)+'/' + str(randomseed) + \
           '/Fold' + str(testfoldInd)
exp_info = 'T_'+str(Ttrain[0])+'_'+str(Ttrain[1])+'_BS_'+str(BATCH_SIZE)+'_HS_'+str(HIDDENSIZE)\
           +'_RHO_'+str(RHO)+'_NF_'+str(Nfea)+'_Buff_'+str(BUFFTIME)+'_Mul_'+str(Nmul)
save_path = os.path.join(exp_name, exp_disp)
out = os.path.join(rootOut, save_path, exp_info) # output folder to save results
# define and load model
Ninv = zTrain.shape[-1] + attrs.shape[-1]

daymet tmean was used!
read usgs streamflow 22.05735492706299
read usgs streamflow 22.18307137489319
read usgs streamflow 19.470875024795532
daymet tmean was used!
read usgs streamflow 22.071636199951172
read usgs streamflow 22.37849497795105


In [1]:
from mm_interface.train_test import CudnnLstmModel
from mm_interface.mm_functional import range_bound_loss

class EnsembleWeights(torch.nn.Module):
    def __init__(self, *, ninv, hiddeninv, drinv=0.5, nmodels=1):
        super(EnsembleWeights, self).__init__()
        self.name = 'EnsembleWeights'
        self.ninv = ninv
        self.nmodels = nmodels

        ntp = nmodels
        self.hiddeninv = hiddeninv

        self.lstminv = CudnnLstmModel(
            nx=ninv, ny=ntp, hiddenSize=hiddeninv, dr=drinv)
        lb_prcp = [0.95]
        ub_prcp = [1.05]
        self.RangeBoundLoss = range_bound_loss(lb=lb_prcp, ub=ub_prcp)

    def forward(self, x, z, prcp_loss_factor):
        z.requires_grad = True

        wghts = self.lstminv(z)
        ntstep = wghts.shape[0]
        ngage = wghts.shape[1]
        wghts_scaled = torch.sigmoid(wghts)  # Change weighting method here.

        prcp_wavg = torch.zeros((ntstep, ngage), requires_grad=True, dtype=torch.float32).cuda()
        prcp_wghts_sum = torch.sum(wghts_scaled, dim=2)
        range_bound_loss_prcp = self.RangeBoundLoss([prcp_wghts_sum], factor=prcp_loss_factor)

        # print(prcp_wavg.shape, wghts_scaled.shape, x.shape, z.shape)

        for para in range(wghts.shape[2]):
            prcp_wavg = prcp_wavg + wghts_scaled[:, :, para] * x[:, :, para]

        x_new = torch.empty((ntstep, ngage, 3), requires_grad=True, dtype=torch.float32).cuda()
        # x_new[:, :, 0] = prcp_wavg
        # x_new[:, :, 1] = x[:, :, self.nmodels]
        # x_new[:, :, 2] = x[:, :, -1]

        # For gradient analysis.
        # grad_daymet = autograd.grad(outputs=wghts_scaled[:, :, 0], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 0]), retain_graph=True)[0]
        # grad_maurer = autograd.grad(outputs=wghts_scaled[:, :, 1], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 1]), retain_graph=True)[0]
        # grad_nldas = autograd.grad(outputs=wghts_scaled[:, :, 2], inputs=z, grad_outputs=torch.ones_like(wghts_scaled[:, :, 2]), retain_graph=True)[0]

        # return x_new, range_bound_loss_prcp, wghts_scaled, grad_daymet, grad_maurer, grad_nldas
        return x_new, range_bound_loss_prcp, wghts_scaled

loading package hydroDL


NameError: name 'torch' is not defined

In [ ]:
model = rnn.prcp_weights(ninv=Ninv, hiddeninv=HIDDENSIZE, prcp_datatypes=len(forType))

# dict only for logging
optModel = OrderedDict(name='LSTM-dPLHBV', nx=Ninv, nfea=Nfea, nmul=Nmul, hiddenSize=HIDDENSIZE, doReLU=True,
                        Tinv=Tinv, Trainbuff=BUFFTIME, routOpt=routing, comprout=comprout, compwts=compwts,
                        pcorr=pcorr, staind=staind, tdlst=tdRep, dydrop=dydrop,buffOpt=buffOpt, TDOpt=TDOpt, ETMod=ETMod)
